# Presentation - Dokumentenverleich - Christoph Stach

Erforderliche Gensim, spaCy und selbst erstellte Hilfsmodule laden.

In [1]:
from gensim.models import Word2Vec, TfidfModel
from gensim.corpora import Dictionary
from gensim.matutils import softcossim
from TextProcessing import lemmatize_word
from database import GetArticles
from random import randrange
from helpers import flatten
from tqdm import tqdm
from validation_data import get_train, get_test
from statistics import mean
import re
import spacy

## Deutsches spaCy Objekt erstellen

Es wird hauptsächlich für das Preprocessing benötigt.

In [2]:
nlp = spacy.load('de')

## Preprocessing definieren

1. Texte von unötigen Sonderzeichen und Textpassagen bereinigen
2. Mit spaCy den Text in viele Sätze aufteilen
3. Mit spaCy sich die Nominalphrasen eines Satzes parsen
4. Stopwörter aus aus den Nominalphrasen entfernen (auf Basis der spaCy Stopwörterlite)
5. Wörter aus den Nonimalphrasen lemmatisieren
6. Wörter in Kleinschreibung umwandeln
7. Alle gefundenen Nominalphrasen eines Dokuments in einem Array speichern

In [3]:
def preprocess(text):
        # Clean text and remove double spacy and special chars and copyrights.
        text = re.sub(' +', ' ', text)
        text = re.sub(r'[^\w äöüÄÖÜß\"\:\,\.?\!\-\_ ]', '', text)
        text = re.sub(' 0 0 ', '', text)
        text = re.sub('Mehr zum Thema\:(.*)', '', text)
        text = re.sub('Berliner Morgenpost [0-9]{4} (.*) Alle Rechte vorbehalten\.', '', text)  # removes copyright if 'Mehr zum Thema' is not in text
        text = re.sub('&', 'und', text)
        text = re.sub(' +', ' ', text)
        text = re.sub(' \.', '.', text)
        text = re.sub('Berliner Morgenpost [0-9]{4} Alle Rechte vorbehalten\.', '', text)
        text = text.strip()

        
        doc = nlp(text)

        sentences = []
        for sentence in doc.sents:  # Split text into sentences
            chunks = []

            for chunk in sentence.noun_chunks:  # Get noun_chunks out of sentence
                tokens = chunk.text.lower().split(' ')
                tokens = list(filter(lambda token: not nlp.vocab[token].is_stop, tokens)) # Remove stopwords
                tokens = list(filter(lambda token: token != '', tokens)) # Remove empty tokens
                tokens = list(map(lambda token: lemmatize_word(token), tokens))  # Lemmatize

                chunk = ' '.join(tokens) # join stopword cleaned noun-chunks together again
                chunk = chunk.lower()  # lower all the text

                if chunk != '':
                    chunks.append(chunk)

            if len(chunks) > 0:  # Only append non empty tokens
                sentences.append(chunks)

        return flatten(sentences)  # Merge words of the sentences together to a document array

## Beispiele

In [4]:
print(preprocess('Eine Beamtin der Mordkommission hatte in der Sendung eine sichergestellte Mütze gezeigt, die der Täter laut Polizei "nachweislich" verlor. Sie fragte, wer den Besitzer kennt. Es handle sich um ein älteres H&M-Frauenmodell, an dem ursprünglich Strasssteine angebracht gewesen seien. Gesucht werden auch Hinweisgeber zu einem blau-weißen Tuch, das dem Opfer gehörte.'))
print(preprocess('Als am Pfingstsonntag Aktivisten leer stehende Häuser in Berlin besetzten, war die Aufregung groß. Während sich die rot-rot-grüne Koalition stritt, wie die Aktion politisch zu bewerten sei, herrschte auch Verwirrung darüber, wie groß das Problem Wohnungsleerstand in Berlin tatsächlich ist. Auf einer Anhörung im Stadtentwicklungsausschuss am Mittwoch wurde diese Wissenslücke geschlossen.'))
print(preprocess('Auch die in die Kritik geratene kommunale „Stadt und Land“, deren seit Jahren leer stehendes Haus in Neukölln am Pfingstsonntag ebenfalls besetzt worden war, verteidigte sich. „Wir haben einen Leerstand von 2,6 Prozent, das entspricht 1154 Wohnungen“, sagte Geschäftsführer Ingo Malter. Knapp ein Prozent stünden sanierungsbedingt leer. „Keine dramatischen Zahlen“, so der „Stadt und Land“-Chef.'))

['beamti', 'mordkommissio', 'sendung', 'sichergestellt mutz', 'tater', 'polizei', 'hinweisgeb']
['pfingstsonntag', 'aktivi', 'leer stehend hau', 'berlin', 'aufregung', 'problem wohnungsleersta', 'anhorung']
['kritik', 'kritik geraten kommunal stadt', 'land', 'leer stehend hau', 'neukoll', 'pfingstsonntag', 'knapp proz', 'dramatisch zahlen']


## Zuvor erstellten Modelle laden

In [5]:
word2vec = Word2Vec.load('./data/noun-chunks/w2v.bin')
dictionary = Dictionary.load('./data/noun-chunks/dict.bin')
tfidf = TfidfModel.load('./data/noun-chunks/tfidf.bin')

## Similarity-Matrix erstellen

Vergleicht alle Wordembeddings von allen Wörtern die im Dictionary vorhanden sind und erstellt ahand dessen eine Similarity-Matrix.
Optional können Gewichte aus einem TfIdf-Model miteinbezogen werden.

In [6]:
similarity_matrix = word2vec.wv.similarity_matrix(dictionary, tfidf)

## Vergleichsfunktion definieren

Vergleicht zwei Texte über die ***soft cosine similarity*** anhand der zuvor erstellten similarty_matrix.

In [7]:
def similarity(text1, text2):
    preprocessed_text1 = preprocess(text1)
    preprocessed_text2 = preprocess(text2)

    bow1 = dictionary.doc2bow(preprocessed_text1)
    bow2 = dictionary.doc2bow(preprocessed_text2)
    
    return softcossim(bow1, bow2, similarity_matrix)

## Beispiel

In [8]:
print(similarity(
    'Als am Pfingstsonntag Aktivisten leer stehende Häuser in Berlin besetzten, war die Aufregung groß. Während sich die rot-rot-grüne Koalition stritt, wie die Aktion politisch zu bewerten sei, herrschte auch Verwirrung darüber, wie groß das Problem Wohnungsleerstand in Berlin tatsächlich ist. Auf einer Anhörung im Stadtentwicklungsausschuss am Mittwoch wurde diese Wissenslücke geschlossen.', 
    'Auch die in die Kritik geratene kommunale „Stadt und Land“, deren seit Jahren leer stehendes Haus in Neukölln am Pfingstsonntag ebenfalls besetzt worden war, verteidigte sich. „Wir haben einen Leerstand von 2,6 Prozent, das entspricht 1154 Wohnungen“, sagte Geschäftsführer Ingo Malter. Knapp ein Prozent stünden sanierungsbedingt leer. „Keine dramatischen Zahlen“, so der „Stadt und Land“-Chef.'
))

0.2636047637143506


## Dublettenfunktion definieren

Vergleicht zwei Texte und gibt True oder False zurück falls die Texte Dubletten darstellen.

In [9]:
def is_duplicate(text1, text2, threshold):
    return similarity(text1, text2) >= threshold

## Beispiel

In [10]:
print(is_duplicate(
    'Als am Pfingstsonntag Aktivisten leer stehende Häuser in Berlin besetzten, war die Aufregung groß. Während sich die rot-rot-grüne Koalition stritt, wie die Aktion politisch zu bewerten sei, herrschte auch Verwirrung darüber, wie groß das Problem Wohnungsleerstand in Berlin tatsächlich ist. Auf einer Anhörung im Stadtentwicklungsausschuss am Mittwoch wurde diese Wissenslücke geschlossen.', 
    'Auch die in die Kritik geratene kommunale „Stadt und Land“, deren seit Jahren leer stehendes Haus in Neukölln am Pfingstsonntag ebenfalls besetzt worden war, verteidigte sich. „Wir haben einen Leerstand von 2,6 Prozent, das entspricht 1154 Wohnungen“, sagte Geschäftsführer Ingo Malter. Knapp ein Prozent stünden sanierungsbedingt leer. „Keine dramatischen Zahlen“, so der „Stadt und Land“-Chef.',
     0.8285802480782033
))

False


## Die Schwelle (threshold) berechnen

Zuerst benötigen wir Lauras trainings und test Daten.

In [11]:
train = get_train()
test  = get_test()

Die Similarity für alle Trainingsdaten bestimmten

In [12]:
duplicates = []
none_duplicates = []

train = get_train()

for data in tqdm(train, desc='Calculating similarities'):
    sim = similarity(data['text1'], data['text2'])
    
    if data['duplicate']:
        duplicates.append(sim)
    else:
        none_duplicates.append(sim)

Calculating similarities: 100%|██████████| 810/810 [02:23<00:00,  5.65it/s]


In [13]:
mean_d = mean(duplicates)
mean_nd = mean(none_duplicates)

print(mean_d)
print(mean_nd)

0.8657823728312567
0.037202124753053435


## Schwelle definieren

Ab dieser Schwelle von Similarity gelten diese Text als Dubletten.

In [14]:
threshold = mean_d - mean_nd

print(threshold)

0.8285802480782033


## Accuracy auf dem Testdatensatz

In [15]:
total = len(test)
correct = 0

for data in tqdm(test, desc='Calculating duplicates'):
    if data['duplicate'] == is_duplicate(data['text1'], data['text2'], threshold):
        correct += 1

Calculating duplicates: 100%|██████████| 69/69 [00:09<00:00,  7.09it/s]


In [16]:
print('Correct: ', correct)
print('Total: ', total)
print('Accuracy: ', (correct / total * 100), '%')

Correct:  56
Total:  69
Accuracy:  81.15942028985508 %


## Accuracy auf dem Traindatensatz

In [17]:
total = len(train)
correct = 0

for data in tqdm(train, desc='Calculating duplicates'):
    if data['duplicate'] == is_duplicate(data['text1'], data['text2'], threshold):
        correct += 1

Calculating duplicates: 100%|██████████| 810/810 [02:22<00:00,  5.70it/s]


In [18]:
print('Correct: ', correct)
print('Total: ', total)
print('Accuracy: ', (correct / total * 100), '%')

Correct:  705
Total:  810
Accuracy:  87.03703703703704 %


## Vergleich zufälliger Texte aus der Datenbank

In [19]:
articles = GetArticles(local=False, test_run=False)

In [20]:
count = len(articles)
rnd1 = randrange(0, count - 1)
rnd2 = randrange(0, count - 1)

text1 = articles[rnd1]['text']
text2 = articles[rnd2]['text']

In [21]:
print(rnd1, text1)
print()
print()
print(rnd2, text2)

  – mit allen Mitteln. Holte pensionierte Finanzbeamte als Laien-Asylentscheider ins Amt, auch Soldaten und Arbeitslose. Das Bamf wuchs von 2000 auf 10.000 Mitarbeiter.  Nur: Viele hatten von Asylrecht oder der Lage etwa in Afghanistan keine Ahnung, wurden "on the job" geschult. Weise gelang es, das träge Bamf zu modernisieren. Heute sagen seine Kritiker: Das passierte auf Kosten der Qualität – und der Sicherheit.   • Jutta Cordt  Anfang 2017 machte Weise Jutta Cordt zur neuen Bamf-Präsidentin. Er kannte sie schon von der Arbeitsagentur. Cordt setzte einerseits auf den umstrittenen Beschleunigungskurs ihres Vorgängers, andererseits konzentrierte sie sich 2017 mehr auf Sicherheit im Asylverfahren und Qualität der Entscheide.   Ihr Vorteil: Mittlerweile kommen deutlich weniger Geflüchtete ins Land. Nur greifen Cordts Maßnahmen zu spät. Das zeigte sich, als sich der Rechtsradikale Franco A. als Syrer ausgab – und Asyl bekam. Das zeigt sich auch im mutmaßlichen Asylbetrug in der Bremer Auß

In [22]:
print(similarity(text1, text2))

0.011434836775954279
